In [1]:
import pandas as pd
import matplotlib.pyplot as plot
import numpy as np

In [2]:
training_labels = pd.read_csv('../Data/training_set_labels.csv')
training_values = pd.read_csv('../Data/training_set_values.csv')
test_values = pd.read_csv('../Data/test_set_values.csv')

In [3]:
training_labels['status_group'] = training_labels['status_group'].map({'non functional': 'non operational', 'functional': 'operational', 
                                     'functional needs repair': 'operational'})

In [4]:
df = training_values.merge(training_labels, on='id')

In [5]:
df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [6]:
df.drop("id", axis = 1,inplace=True)

In [7]:
def valuecounts(column, df):
    print(column)
    return df[column].value_counts()

In [8]:
for column in df.columns:
    print(valuecounts(column, df))

amount_tsh
0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64
date_recorded
2011-03-15    572
2011-03-17    558
2013-02-03    546
2011-03-14    520
2011-03-16    513
             ... 
2011-09-01      1
2011-09-26      1
2011-09-18      1
2012-01-25      1
2011-09-20      1
Name: date_recorded, Length: 356, dtype: int64
funder
Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
                          ... 
Japan Government             1
Kilol                        1
Maseka Community             1
Muivaru                      1
Fosecu                       1
Name: funder, Length: 1897, dtype: int64
gps_height
 0       20438
-15         60
-16         55
-13         55
-20         52
         ...  

Name: quantity, dtype: int64
quantity_group
enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64
source
spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64
source_type
spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64
source_class
groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64
waterpoint_type
communal standpipe             28522
hand pump                      17488
other                   

In [9]:
columns_to_drop = ["amount_tsh", "num_private", "recorded_by", "payment_type", "extraction_type", "extraction_type_group", 
                   "water_quality", "quantity_group"]
df_small = df.drop(columns_to_drop, axis = 1)

In [10]:
df_small_small = df_small.replace({'other': None,'none': None,'unknown' : None, -2.00E-08: None, "0": None})
df_small_small[["district_code","population","construction_year"]].replace({0: None}, inplace=True)

C:\Users\rober\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [11]:
df_small_small.dropna(subset=['latitude'],inplace=True)

In [12]:
construction_year
district_code 
population 

NameError: name 'construction_year' is not defined

In [ ]:
df_small_small.isna().sum()

In [ ]:
df_small_small2.dropna(subset=['latitude'],inplace=True)

In [ ]:
df_small_small2.isna().sum()

In [ ]:
for column in df_small_small.columns:
    print(valuecounts(column,df_small_small))